# READ_EDAB_Utilities_py Tutorial
* This tutorial shows how the functions in READ_EDAB_Utilities_py can be used to work with spatial data

In [44]:
# Import necessary packages

# for working with shapefiles
import geopandas as gpd

# for working with netcdfs
import xarray as xr

# READ_EDAB functions
import READ_EDAB_Utilities_py as reupy

# for subsetting data to shapefile regions
import regionmask

# for math
import pandas as pn

In [3]:
# Load some test data
ds = xr.open_dataset('test_data/GLORYS_daily_BottomSalinity_2019.nc')

In [4]:
# Take a look at it
ds

<xarray.Dataset>
Dimensions:    (longitude: 193, latitude: 157, time: 365)
Coordinates:
  * longitude  (longitude) float64 -78.0 -77.92 -77.83 ... -62.17 -62.08 -62.0
  * latitude   (latitude) float64 47.0 46.92 46.83 46.75 ... 34.17 34.08 34.0
  * time       (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-12-31
Data variables:
    BottomS    (time, latitude, longitude) float32 ...
    crs        int32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and terra (version 1.2-5)
    date:         2023-11-29 15:34:42

In [7]:
# Load in the shapefile
epu = gpd.read_file('test_data/EPU_NOESTUARIES.shp')

In [9]:
# Take a look at it
epu

,FID_MAB_fu,ID,GRIDCODE,FID_GoM_fu,ID_1,GRIDCODE_1,FID_SS_ful,ID_12,GRIDCODE_2,FID_GB_ful,ID_12_13,GRIDCODE_3,EPU,Shape_Leng,Shape_Area,geometry
0,0,4.0,4.0,-1,0.0,0.0,-1,0.0,0.0,-1,0.0,0.0,MAB,2.352719e+06,8.813609e+10,"POLYGON ((-70.5 41.16667, -70.5 41, -70.5 40.8..."
1,-1,0.0,0.0,0,2.0,7.0,-1,0.0,0.0,-1,0.0,0.0,GOM,1.240777e+06,5.393245e+10,"POLYGON ((-67.5 42, -68 42, -68 41.83333, -68...."
2,-1,0.0,0.0,-1,0.0,0.0,0,1.0,6.0,-1,0.0,0.0,SS,1.169825e+06,2.549858e+10,"POLYGON ((-66 41.66667, -66 41.83333, -66.1666..."
3,-1,0.0,0.0,-1,0.0,0.0,-1,0.0,0.0,0,3.0,5.0,GB,1.231414e+06,4.914739e+10,"POLYGON ((-66 41.66667, -66 41.33333, -66.1666..."


In [14]:
# Get each EPU

mab = epu[epu.EPU == 'MAB']
gom = epu[epu.EPU == 'GOM']
ss = epu[epu.EPU == 'SS']
gb = epu[epu.EPU == 'GB']

In [15]:
# Subset the salinity data for each EPU

mab_salt = reupy.crop_nd(ds, 'longitude', 'latitude', mab)
gom_salt = reupy.crop_nd(ds, 'longitude', 'latitude', gom)
ss_salt = reupy.crop_nd(ds, 'longitude', 'latitude', ss)
gb_salt = reupy.crop_nd(ds, 'longitude', 'latitude', gb)

In [53]:
# Get the mean salinity value in each EPU where salinity is between 30 and 33
mab_mean = mab_salt.BottomS.where((mab_salt.BottomS >= 30) & (mab_salt.BottomS <= 33)).mean()
gom_mean = gom_salt.BottomS.where((gom_salt.BottomS >= 30) & (gom_salt.BottomS <= 33)).mean()
ss_mean = ss_salt.BottomS.where((ss_salt.BottomS >= 30) & (ss_salt.BottomS <= 33)).mean()
gb_mean = gb_salt.BottomS.where((gb_salt.BottomS >= 30) & (gb_salt.BottomS <= 33)).mean()

In [55]:
# Make a table of the data
pn.DataFrame({'EPU': ['MAB','GOM','SS','GB'],
              'mean bottomS': [mab_mean.values, gom_mean.values, ss_mean.values, gb_mean.values]})

,EPU,mean bottomS
0,MAB,32.217537
1,GOM,32.502907
2,SS,32.09776
3,GB,32.018238
